# Sampling from discrete distributions

In [107]:
import time
import bisect
import random
import numpy as np
import functools as ft
import plotly.offline as py
from scipy.optimize import curve_fit
from concurrent.futures import ProcessPoolExecutor
from plotly.graph_objs import Histogram, Bar, Scatter, Figure, Layout

py.init_notebook_mode(connected=True)

%load_ext cython

## Sampling

We sample from a randomly generated probability mass function (pmf) using different algorithms, then we compare with the theoretical distribution.

In [36]:
%%cython

# Using Cython here to optimize the samplers.

import bisect
import numpy as np
cimport numpy as np
from libc.stdlib cimport rand, RAND_MAX


cdef class Sampler:
    cpdef np.ndarray sample(self, int size=1):
        cdef np.ndarray samples = np.empty(size, dtype=np.float_)
        cdef int i
        for i in range(size):
            samples[i] = self._sample()
        
        return samples

cdef class AcceptRejectSampler(Sampler):
    
    cdef np.ndarray p
    cdef unsigned int n_items
    cdef double p_max

    def __cinit__(self, np.ndarray p):
        self.p = p
        self.n_items = len(p)
        self.p_max = p.max()
        
    
    cpdef int _sample(self):
        cdef unsigned int i
        cdef double r

        while True:
            # Choose one element randomly
            i = int(rand()/RAND_MAX * self.n_items)

            # Accept/reject
            r = rand()/RAND_MAX * self.p_max
            
            if r < self.p[i]:
                return i
            

    
cdef class TowerSampler(Sampler):
    
    cdef np.ndarray p
    cdef np.ndarray c

    def __cinit__(self, np.ndarray p):
        self.p = p
        self.c = np.cumsum(p, dtype=np.float_)
    
    cpdef int _sample(self):
        cdef double r = rand()/RAND_MAX
        cdef unsigned int mid, lo = 0, hi = len(self.c)
        
        while lo < hi:
            mid = (lo + hi) // 2
            if r > self.c[mid]:
                lo = mid + 1
            else:
                hi = mid

        return lo

In [37]:
def generate_uniform_pmf(size=5):
    r = np.random.random(size)

    return r / np.sum(r)

In [38]:
n_items = 5
p = generate_uniform_pmf(n_items)


ar = AcceptRejectSampler(p)
ts = TowerSampler(p)

data = [
    Bar(y=p, name="Theoretical probability"),
    Histogram(x=ar.sample(10000), histnorm="probability", name="Accept/Reject sampling"),
    Histogram(x=ts.sample(10000), histnorm="probability", name="Tower sampling")
]

lyt = Layout(title="Comparison of sampling algorithms", xaxis=dict(title="Item"),
             bargroupgap=0.1)

py.iplot(Figure(data=data, layout=lyt))

## Computational cost

It’s useful to create a class that performs the various benchmarks. We use multi-process concurrency to perform the benchmark faster.

In [32]:
class SamplingBenchmark(object):

    def __init__(self, pmf_generator, samplers, sample_size=100000, ns=None, num_pmfs=1):
        """Benchmark a collection of samplers.
        
        Positional agruments:
        pmf_generator -- function which generates a pdf given n_items
        samplers -- a list of Sampler classes to benchmark

        Keyword arguments:
        name -- name of the plot file
        sample_size -- size of the sample extracted from the samplers
        ns -- list of different n_items to benchmark
        num_pmfs -- the number of pmfs for each value of n_items
        """
        self.pmf_generator = pmf_generator
        self.samplers = samplers
        self.sample_size = sample_size
        self.num_pmfs = num_pmfs

        if ns is None:
            self.ns = np.concatenate((
                np.arange(1, 10, step=1),
                np.arange(10, 1000, step=10),
                np.arange(1000, 10000, step=100),
                np.arange(10000, 20000, step=200),
            ))
        else:
            self.ns = ns

        self.pmfs = np.empty((len(self.ns), self.num_pmfs), dtype=object)
        self.results = {}


    def run(self):
        """Benchmark the samplers and return a list of execution times."""
        self._init_pmfs()

        with ProcessPoolExecutor() as executor:
            for sampler in self.samplers:
                fn = ft.partial(self._benchmark, sampler=sampler)
                times = executor.map(fn, self.pmfs)

                self.results[sampler.__name__] = list(times)

        return self.results

    def plot(self):
        """Plot the execution times."""
        data = []
        
        for sampler in self.results:
            data.append(Scatter(x=self.ns, y=self.results[sampler], name=sampler))

        py.iplot(data)

    
    def _init_pmfs(self):
        for i, n_items in enumerate(self.ns):
            for j in range(self.num_pmfs):
                self.pmfs[i][j] = self.pmf_generator(n_items)

    def _benchmark(self, pmfs, sampler):
        start_time = time.process_time()
        for pmf in pmfs:
            s = sampler(pmf)
            s.sample(self.sample_size)

        return (time.process_time() - start_time) / self.num_pmfs

In [90]:
# Define our constants
SAMPLERS = [AcceptRejectSampler, TowerSampler]
SAMPLE_SIZE = 100000

### Accept/reject sampling for uniformly generated pmfs
For estimating the computational cost of the accept/reject method, we can model it as a Bernoulli process. Each time we draw a sample with constant cost $c$ (independent of the number of items $N$) and we accept it with probability $p_{acc}$, otherwise we repeat. The cost will be $c T$ where $T$ is a random variable indicating the number of iterations until the first successful sample.

We can calculate $p_{acc}$ as the ratio between the area occupied by the probability mass function and the total area in which we draw the sample. If we restrict the total area to $max(p_i)$ and we consider columns of width 1 we get:

$$
\begin{align*}
&\mathcal{A}_{tot} = max(p_i) \cdot N = \frac{max(r_i)}{\sum_{i=1}^{N}{r_i}} \cdot N \\
&\mathcal{A}_{dist} = 1  \mathrm{\ by\ normalization} \\
&p_{acc} = \frac{\mathcal{A}_{dist}}{\mathcal{A}_{tot}} = \frac{ \sum_{i = 1}^{N}{r_i}}{max(r_i)  \cdot N} 
\end{align*}
$$

For N large we have

$$
\begin{align*}
&max(r_i) \sim 1
&\sum_{i=1}^{N}{r_i} \sim N
\end{align*}
$$

Using the fact that $T$ is geometrically distributed
$$
\mathbb{E}[T] = \frac{1}{p_{acc}} = \frac{max(r_i)  \cdot N}{\sum_{i = 1}^{N}{r_i}} \sim 1
$$

Thus the execution time is $O(1)$ for large $N$.

Note: this is just theory, while in the following code we are measuring real world performance (timing the process) and many different conditions will influence the measurement (e.g. memory, I/O, load, …). We actually expect a slowly increasing execution time because the python interpreter has to deal with larger and larger arrays, also we will not be surprised if strange artifacts like unexpected jumps appear in the plots. Yet the aymptotic behaviour we obtained theoretically is clearly recognizable in the plots below, this means that, despite the noise, our theoretical tools get to the heart of the matter.


### Tower sampling for uniformly generated pmfs

Since we are considering big sample sizes, we can neglect the $O(N)$ cost of computing the cumulative pmf which is done only once. Again we can consider the cost of drawing a random number $r$ constant. The bottleneck of the algorithm is instead recognizing which interval contains $r$. This can be done in $O(\log N)$ if using binary search.

For small $N$, depending on the fine optimization of the code this algorithm may outperform the accept/reject method since it draws a single random number with acceptance 1 and just repeat the loop at most $\log_2 N$ times. But in the long run, if we consider pmfs generated from a uniform distribution, the accept/reject algorithm will be faster. Note that this does not hold in general but is strongly linked with the fact that the maximum is $O(1)$ (this will be true alse for other ditributions that have a bounded domain).

In [34]:
bm_uniform = SamplingBenchmark(generate_uniform_pmf, SAMPLERS, sample_size=SAMPLE_SIZE)
res_uniform = bm_uniform.run()  # takes some time…
bm_uniform.plot()

### Accept/reject sampling for exponential distribution

For pmfs generated from an exponential distribution, we can repeat the procedure above in a similar way, just by changing the order of $max(r_i)$ that is now $O(\log N)$ instead of $O(1)$. We easily obtain that the computational cost is then $O(\log N)$. In this case accept/reject and tower sampling have comparable cost and differences are to be acribed to different code optimization (actually accept/reject will probably perform slightly better since it does not compute the cumulative mass function). 

We can cautiously try to generalize by saying that for pmfs generated from thin tailed distributions the cost of the accept/reject algorithm will scale as the maximum of the distribution.

In [39]:
def generate_exponential_pmf(size=5):
    r = -np.log(np.random.random(size))

    return r / np.sum(r)

bm_exponential = SamplingBenchmark(generate_exponential_pmf, SAMPLERS, sample_size=SAMPLE_SIZE)
bm_exponential.run()  # takes even more time…
bm_exponential.plot()

### Accept/reject for Pareto distribution


If $R_i = U^{-\frac{1}{2}}$ where $U$ is a uniform random variable in [0, 1) is equivalent to say $r_i$ are drawn from the Pareto distribution:

$$
p(r) = \frac{2}{r^3}, r \in [1, \infty)
$$

This is a heavy tailed distribution with infinite variance. The maximum $max(r_i)$ scales as a power law, in this case we expect

$$
max(r_i) \sim N^{\frac{1}{2}}
$$

The sum will still scale (at first order) as N, thus we can repeat the derivation above (the generalization we made still holds), obtaining:

$$
\mathbb{E}[T] \sim N^{\frac{1}{2}} 
$$

Since we are dealing with an heavy tailed distribution, benchmarking a single pmf for each $N$ would lead to spikes in our measurement corresponding to events where the maximum is very big. We try to avoid this by averaging over 100 pmfs for each value of $N$.  

In [96]:
def generate_pareto_pmf(size=5):
    r = 1. / np.sqrt(np.random.random(size))

    return r / np.sum(r)

# We use wider steps for n_items than the default since this is the slowest algorithm.
ns = np.concatenate((
        np.arange(1, 10, step=1),
        np.arange(10, 1000, step=25),
        np.arange(1000, 10000, step=500),
        np.arange(10000, 20000, step=1000),
    ))

# Average on 100 pmfs and reduce proportionally the sample size to avoid too heavy computation.
bm_pareto = SamplingBenchmark(generate_pareto_pmf, SAMPLERS, sample_size=SAMPLE_SIZE//100, num_pmfs=100, ns=ns)
results = bm_pareto.run()  # now wait for last year
bm_pareto.plot()


To verify our assumptions we can make a fit with a power law. If we get an exponent close to 0,5 our analysis is somewhat confirmed—or at least, not rejected.

In [103]:
f = lambda x, a, b: a*(x**b)

fit = curve_fit(f, ns, results["AcceptRejectSampler"])

a, b = fit[0]
print("Fitting result for f(x) = a*x^b\n"
      "a = {:f}\nb = {:f}".format(a, b))

if abs(b - 0.5) < 0.05:
    print("Not bad!!!")

data = [Scatter(x=ns, y=results["AcceptRejectSampler"], name="Data"),
        Scatter(x=ns, y=[f(n, a, b) for n in ns], name="Fit".format(a, b))]
py.iplot(Figure(data=data, layout=Layout(title="Fit of the accept/reject sampler cost")))

Fitting result for f(x) = a*x^b
a = 0.000147
b = 0.501393
Not bad!!!


## Conclusions

The accept/reject sampling is very effective in certain cases, for example if we generate the pmf from a uniform distribution, whilst in other cases it may fail miserably (e.g. when we generate the pmf using heavy tailed distributions). The tower sampling is more robust: even if it is slower in some cases, it has a cost $O(\log N)$ regardless of the distribution.

In conclusion, if one has to choose blindly between the two sampling algorithms, tower sampling is the wisest choice. That said, one should check whether the accept/reject has better performances in the specific case. 